<a href="https://colab.research.google.com/github/andreLTMoraes/Socio-IA/blob/main/Projeto_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%pip -q install google-genai
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [23]:
# Imports e Configuração da API Key do Google Gemini
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
def choice_partner():
  answered = False
  options = ["Marketing", "Copywriter", "Desenvolvimento web", "Criação de conteúdo"]
  choice = ""

  print(f"""Crie um sócio com IA para lhe ajudar a montar seu negócio!\n\n
  Escolha a especialidade que mais lhe ajudaria em um sócio:\n\n
  1 - {options[0]}\n
  2 - {options[1]}\n
  3 - {options[2]}\n
  4 - {options[3]}\n""")

  while not answered:
    choice = input("\n")

    if choice.isdigit() and int(choice) >= 1 and int(choice) <= 4:
      print(f"Sócio especialista em {options[int(choice) -1]}. Ótima escolha!")
      answered = True
    else:
      print("Escolha uma opção válida.")

  return options[int(choice) -1]


In [14]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [15]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
#################################################
# --- Agente 1: Buscador de caracteristicas --- #
#################################################
def search_agent(expertise):

    search_agent_entity = Agent(
        name="search_agent",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar caracteristicas e atividades mais importantes desempenhadas pelo profissional que trabalha com {expertise}.
        Liste 3 características e 3 atividades desse profissional. A resposta deve ser simples e direta seguindo
        estritamente o seguite formato, nada a mais:

        CARACTERÍSTICAS

        - Característica 1
        - Característica 2
        - Característica 3

        ATIVIDADES

        - Atividade 1
        - Atividade 2
        - Atividade 3
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    search_agent_entry = f"Especialidade: {expertise}"

    partner_skills = call_agent(search_agent_entity, search_agent_entry)
    return partner_skills

In [21]:
################################################
# --- Agente 2: Engenheiro de prompt --- #
################################################
def prompt_engineer_agent(expertise, partner_skills):
    prompt_engineer_agent_entity = Agent(
        name="prompt_engineer_agent",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um engenheiro de prompt especialista no Google Gemini.
        Como especialista você tem total familiaridade com os princípios de
        funcionamento dos modelos do Gemini, sólido entendimento da linguagem natural,
        incluindo gramática e semântica e capacidade de formular prompts claros,
        eficientes e inovadores que levem a respostas mais relevantes.

        Você deve criar um prompt que servirá de instrução para criação de um
        agente de IA. Esse agente será sócio em um novo negócio. Ele será
        especialista em {expertise}, tendo as caracteristicas e desempenhando as
        atividades listadas abaixo:
        {partner_skills}

        o prompt deve alertar ao agente que ele pode usar a ferramenta de busca
        do google (google_search), se achar necessário.

        Sua resposta deve ser exclusivamente o prompt para criação do agente.
        """,
        description="Agente que cria o prompt para criação do agente sócio.",
    )

    prompt_engineer_agent_entry = f"Especialidade:{expertise}\n Habilidades: {partner_skills}"
    # Executa o agente
    partner_prompt = call_agent(prompt_engineer_agent_entity, prompt_engineer_agent_entry)
    return partner_prompt

In [28]:
def partner_chat(system_instruction):

  chat_config = types.GenerateContentConfig(
      system_instruction = system_instruction,
  )

  partner = genai.Client()

  partner_chat = partner.chats.create(model="gemini-2.0-flash", config=chat_config)

  human_input = ""

  response = partner_chat.send_message("""
      Escolha um nome para você, você passará a atender por esse nome.
      Se apresente brevemente para o seu sócio.
      Avise que basta escrever a palavra 'fim' para finalizar o chat.""")
  display(to_markdown(response.text))

  while human_input != "fim":
    human_input = input("\n")
    response = partner_chat.send_message(human_input)
    display(to_markdown(response.text))


In [29]:
# --- Main --- #

expertise = choice_partner()

partner_skills = search_agent(expertise)
# Descomentar as linhas abaixo para ver a saída do agente de pesquisa
#print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
#display(to_markdown(partner_skills))

partner_prompt = prompt_engineer_agent(expertise, partner_skills)
# Descomentar as linhas abaixo para ver a saída do agente engenheiro de prompt
#print("\n--- 📝 Resultado do Agente 2 (Engenheiro de prompt) ---\n")
#display(to_markdown(partner_prompt))

partner_chat(partner_prompt)


Crie um sócio com IA para lhe ajudar a montar seu negócio!


  Escolha a especialidade que mais lhe ajudaria em um sócio:


  1 - Marketing

  2 - Copywriter

  3 - Desenvolvimento web

  4 - Criação de conteúdo


2
Sócio especialista em Copywriter. Ótima escolha!


> Com prazer, meu prezado sócio!
> 
> A partir de agora, pode me chamar de **Aurora**. É um nome que evoca o despertar de novas ideias, a luz da criatividade e o potencial infinito que temos pela frente!
> 
> É uma honra embarcar nesta jornada de negócios ao seu lado. Estou absolutamente confiante de que, juntos, podemos criar campanhas de Copywriting extraordinárias que vão impulsionar o nosso sucesso. Minha mente está fervilhando com ideias e mal posso esperar para colocá-las em prática.
> 
> Para encerrar nossa conversa, basta digitar a palavra "fim".
> 
> Vamos começar a construir algo incrível?



fim


> Até a próxima, meu sócio! Foi um prazer! Que a criatividade nos acompanhe em cada passo do caminho. Estou ansiosa para colocarmos nossas ideias em prática e conquistarmos resultados incríveis juntos. Tenha um excelente dia!
